In [1]:
#Cargamos las librerias
#Carga de librerias
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score

# Versiones anteriores a 1.2 de sklearn: from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn import tree

In [2]:
#Cargamos los datos de Y
Y = pd.read_csv('./../../DatosProcesados/Y_train_V1.csv', sep=',', encoding = "ISO-8859-1")
Y.shape


(7802, 1)

In [3]:
#Cargamos los datos de X
X = pd.read_csv('X_train_V2.csv', sep=',', encoding = "ISO-8859-1")
X.shape

(7802, 12124)

In [4]:
# Dividir los datos en entrenamiento y test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

Y_train = Y_train['Class']

In [5]:
X_train.shape, X_test.shape

((6241, 12124), (1561, 12124))

In [7]:
#Creamos los elementos necesarios para calibrar el mejor modelo:
particiones = KFold(n_splits=5, shuffle=True, random_state = 1)
profundidades = [i for i in range(20,30)]
caminos = [i for i in range(112,116)]

In [8]:
param_grid = {'max_features':caminos,'max_depth':profundidades}

In [9]:
forest = RandomForestClassifier(random_state=111)

In [10]:
mejor_modelo = GridSearchCV(forest, param_grid, cv=particiones)

In [11]:
mejor_modelo.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=RandomForestClassifier(random_state=111),
             param_grid={'max_depth': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
                         'max_features': [112, 113, 114, 115]})

In [12]:
mejor_modelo.best_params_

{'max_depth': 29, 'max_features': 112}

In [13]:
forest_final = mejor_modelo.best_estimator_

In [14]:
# Probemos ahora este modelo sobre test.
y_pred_train = forest_final.predict(X_train)

In [15]:
#Sacamos las importancias
importancia= forest_final.feature_importances_
importancia
importancia_atributo = pd.DataFrame(data={"Atributo": X_train.columns,"Importancia": importancia})
importancia_atributo = importancia_atributo.sort_values(by='Importancia', ascending=False).reset_index(drop=True)
importancia_atributo

,Atributo,Importancia
0,excelente,0.015602
1,mal,0.012333
2,mala,0.010425
3,hotel,0.009570
4,habitaciÃ³n,0.006926
...,...,...
12119,factorÃ­a,0.000000
12120,clubes,0.000000
12121,emblematicos,0.000000
12122,clÃ¡sicas,0.000000


In [16]:
#Analisis cuantitivo
y_pred_test = forest_final.predict(X_test)
print('Exactitud sobre entrenamiento: %.2f' % accuracy_score(Y_train, y_pred_train))
print('Exactitud sobre test: %.2f' % accuracy_score(Y_test, y_pred_test))

Exactitud sobre entrenamiento: 0.74
Exactitud sobre test: 0.40


In [17]:
print(classification_report(Y_test, y_pred_test)) 

              precision    recall  f1-score   support

           1       0.64      0.11      0.18       150
           2       0.42      0.27      0.33       235
           3       0.35      0.16      0.22       317
           4       0.35      0.15      0.21       389
           5       0.41      0.93      0.57       470

    accuracy                           0.40      1561
   macro avg       0.43      0.32      0.30      1561
weighted avg       0.41      0.40      0.34      1561



In [18]:
# Get unique values and their counts
unique_values, counts = np.unique(y_pred_test, return_counts=True)

# Print the counts
for value, count in zip(y_pred_test, counts):
    print(f"{value}: {count}")

5: 25
2: 154
5: 147
5: 169
4: 1066


In [ ]:
index = np.where(y_pred_train == 1)[0]